# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [7]:
chromedriver = 'C:/ProgramData/Anaconda3/Lib/site-packages/chromedriver.exe'
browser = webdriver.Chrome(executable_path = chromedriver)
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")


In [8]:
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

In [9]:
html_source = browser.page_source

browser.close()

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [10]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [17]:
# 爬取網頁內容，解析後萃取新聞摘要
#html = browser.page_source
html = html_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 蔡司在台擴大招募近百名員工　打造「眼視健康照護」方案

▲左起為蔡司台灣半導體光罩解決方案業務處長葉怡君、蔡司台灣總經理章平達、德國在台協會副處長陶艾瑪。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
百年光學領導品牌德國蔡司今（27）日宣布，今年正式在台北設置新辦公室，打造「眼視健康照護﹙Eye Health Care﹚」方案，並在視力保健、醫療技術、半導體領域開出近百職缺，積極招募專業人才，推動相關領域發展。
根據中華民國眼科醫學會於去年公布的「2019全台護眼趨勢調查」結果指出，國人每日3C用眼時間逐年增加，去年已高達10.7小時，年輕族群使用行動裝置的用眼時間更為驚人，有高達1/3的年輕人每天使用小螢幕觀看網路影片高達3小時，民眾的視力健康更是不容忽視。
▲蔡司台灣總經理章平達。（圖／記者姚惠茹攝）
▲左起為蔡司台灣半導體光罩解決方案業務處長葉怡君、蔡司台灣總經理章平達、德國在台協會副處長陶艾瑪。（圖／記者姚惠茹攝）
蔡司說明，面對人口老化進展快速，蔡司推出眼睛健康初檢服務，運用蔡司最新科技與儀器，透過與專業醫師和專業驗光師的結合，打造「蔡司視力保健生態圈」，為民眾眼睛進行篩檢，協助民眾有機會早期發現可能的眼疾問題。
蔡司補充，近年積極推動的蔡司優視力體驗，攜手專業驗光師與眼鏡業者，根據台灣現況制定標準驗光流程，推動專業驗光儀器，以現代化與數位化的驗光空間與驗光流程，為國人守護健康好視力，目前全台已有36間「蔡司優視力專家」眼鏡店，搭載完整蔡司專業數位驗光儀器。
▲蔡司打造全方位的眼睛健康管理解決方案。（圖／蔡司提供）
蔡司台灣總經理章平達表示，蔡司秉持著在地化與專業化的方向深耕台灣市場，面對大環境的挑戰，針對高近視率、人口老化、驗光流程專業化，蔡司台灣分別推出專屬的解決方案，包括蔡司全方位近視管理解決方案、蔡司眼睛健康檢測系統與蔡司優視力體驗，以提升民眾好視力。
章平達宣布，蔡司台灣在視力保健、醫療技術、半導體領域開出近百職缺，積極招募專業人才，推動相關領域發展，今年更將在台灣建立全球零件中心，並預計2021年於台南開設教育訓練中心，期待最新的半導體微影技術﹙EUV﹚成果。
▲蔡司辦公室「顧客服務中心」。（圖／

▲「遠傳搶付」創新商業模式今（27）日正式上線。（圖／記者陳心怡攝）
記者陳心怡／台北報導
遠傳創新平台結合跨業品牌力量，讓消費者可在「遠傳搶付」官網、及合作夥伴的實體或線上門市，將自己心儀的商品、適合的門號方案搭配組合，即可享有遠傳優質的網路，結合各品牌商品或服務；合作夥伴也可透過「遠傳搶付」簡易的操作後台，輕鬆上架產品以及服務，與遠傳電信服務共建電信門號產品組合，達到一費雙享三贏局面。
遠傳電信總經理井琪表示：｢遠傳近年推動企業轉型，讓電信本業再出發。推出的『遠傳搶付』具備三大創新面向，包含打造全新商業模式、真正多元的電信門號方案、與跨業夥伴開放式合作。透過數位平台策略結盟異業品牌，滿足不同族群的消費者需求，打造最完善的電信行動生活圈。｣
「遠傳搶付」打造門號申辦全新玩法，消費者只需在合作夥伴的實體店面，可通過現場QR code掃碼，或品牌官網、線上入口進入品牌專屬的「遠傳搶付」頁面，再透過線上申辦門號，方案包含月付399/699 4G資費，以及999/1,399 5G資費，申辦成功後即可獲得依據不同方案對應的專屬兌換序號，享有專屬的電信門號組合產品。
以申辦月付1,399方案為例，搭配肌膚保養產品，即可獲得相當於價值三萬的天然護膚保養產品組。
----------------------------------------------------------------------

[13] 7-11進軍河南！開幕日「銷售額277萬」...刷新全球單店「單日銷售紀錄」

▲大陸河南省首間7-ELEVEN23日開幕。（圖／翻攝自微博／新浪河南）
實習記者高琳景／綜合報導
便利商店是台灣民眾在生活上不可或缺的生活機能，但在大陸卻仍不普遍。近日河南鄭州第一間7-ELEVEN終於開幕，沒想到當日銷售額竟刷新全球7-11單店紀錄，達到65萬人民幣（約新台幣277萬）。
根據《新浪河南》報導，鄭州第一家7-11於23日開幕，引發盛況空前的排隊人潮，甚至還有許多人慕名而來「拍照打卡」，以「一個傳說中的網紅超市」來介紹7-ELEVEN。這家7-ELEVEN開幕第一天提供了5000個限量福袋，許多人都是為了福袋和超值優惠而來。
而根據《東方今報》報導，鄭州這家7-ELEVEN開幕第一天的單日銷售額，竟然高達到65萬人民幣（約277萬新台幣），刷新全球7-ELEVEN單一店面的

▲國發會公布9月燈號持續維持綠燈。（圖／國發會提供）
記者余弦妙／台北報導
國家發展委員會今（27）日公布9月景氣對策信號綜合判斷分數為27分，較上月增加1分，燈號續呈綠燈；景氣領先、同時指標持續上升，反映國內經濟逐漸趨穩。
根據調查結果顯示，9項構成項目中，批發、零售及餐飲業營業額由黃藍燈轉呈黃紅燈，分數增加2分；工業生產指數、製造業銷售量指數皆由綠燈轉呈黃紅燈，分數各增加1分；股價指數由紅燈轉呈黃紅燈、海關出口值由黃藍燈轉呈藍燈、機械及電機設備進口值由綠燈轉呈黃藍燈，分數各減少1分；其餘3項燈號不變。
國發會表示，受惠各國經濟活動陸續恢復，終端需求回溫，以及美國對陸廠禁令生效前的提前備貨效應，9月以美元計價之出口持續擴增，生產、銷售指標亦持續成長；加以振興三倍券、節慶帶動買氣，9月零售及餐飲業營業額持續擴增。
展望未來，國發會指出，近期國際主要預測機構陸續調升我國經濟成長表現，且消費性電子新品相繼推出，加上新興科技應用需求拉抬，均有助出口回穩。此外，「投資臺灣三大方案」效益持續發酵，政府加速執行公共建設等，國內投資動能延續，有助整體經濟持穩；不過，全球武漢肺炎疫情有再次升溫疑慮，加以中美爭端未解，需密切關注後續發展，妥為因應。
----------------------------------------------------------------------

[21] 安力-KY打進日系及美系遊戲機大廠　董座許振焜：明年佔比可望達10%

▲安力-KY董事長許振焜。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
散熱廠安力-KY董事長許振焜今（27）日表示，今年正式打進日系及美系遊戲機大廠，已經從5月開始小量出貨，並在8、9月達到出貨最大量，累積今年前9月營收達7300萬元，佔整體營收達5%，預計明年穩定供貨佔比可望達10%。
許振焜表示，安力-KY散熱關鍵零件的終端應用產品以NB為主，今年受到疫情的影響，帶動安力-KY的業績大增，並順利打進日系及美系家用遊戲機大廠，已從5月開始小量出貨，並在8、9月達到出貨最大量，累積今年前9月營收達7300萬元，佔整體營收達5%。
許振焜指出，安力-KY的主要產品NB佔整體營收達85%，非NB的部分遊戲機佔5％、消費電子佔3%，其它7%則是車用、通訊、基地台、居家等產品，並在今年增加華為、立訊等新客戶，預計明年隨著散熱

▲「類出國」新玩法，雄獅與星夢郵輪再推「環島郵輪」。（圖／雄獅旅遊提供）
記者林淑慧／台北報導
新冠肺炎疫情衝擊觀光產業，全球唯一的實體大型旅展「2020 ITF台北國際旅展」本周五將登場，雄獅集團逆勢擴大參展攤位，推出疫後全球首度的「環島郵輪」限定航次最高五折優惠，全面搶攻國旅商機。
ITF是全年規模最大的旅展，吸客效益備受矚目，台灣觀光協會表示，今年旅宿業者在ITF現場推出獨家最殺優惠，要徹底滿足民眾國旅需求，力拚來客數、業績與去年持平。
▲雄獅旅行社董事長王文傑。（圖／記者湯興漢攝）
雄獅品牌策略總經理游國珍表示，政府紓困3.0計劃中，針對觀光業的薪資補助僅至九月底為止，身為上市櫃公司的雄獅集團積極開源節流，力拚國內觀光，希望每月仍能保持4億元營收，但仍與往年平均月營收25億元差距甚大。
雄獅集團今年躍居為旅展規模最大業者，以國旅主題區分參展攤位，包括：郵輪、鐵道、森林、單車、大人囝仔、票券、自由行等多個主題「館中館」方式呈現，並進駐雄獅的餐飲事業gonna、小獅堂，銷售餐飲禮券及特色飲品等。
另外，看準「類出國」正夯，雄獅旅遊與星夢郵輪在交通部的支持下，推出疫後全球首度的「環島郵輪」，預計將停靠基隆、台南、澎湖、高雄、花蓮等熱門旅遊目的地搶市。
游國珍指出，雄獅推出｢探索夢號｣國內環島5天4夜，旅客可搭乘國際郵輪完成環島之旅，用不同的視角探索台灣之美，旅展現場推出「環島郵輪大樂透」活動，現場報名可享優惠驚喜價，限定航次最高五折優惠。
----------------------------------------------------------------------

[34] 廣達攜手健保署推動智慧醫療　林百里：將成台灣下一個「護國神山」

記者姚惠茹／台北報導
廣達（2382）董事長林百里今（27）日參與健保署舉辦的「健保資料人工智慧應用研討會」，談到台灣發展AI、大數據，結合5G之後，將可望帶動智慧醫療、精準醫療成為台灣下一個「護國神山」產業。
健保署表示，為擴充台灣醫療AI發展量能，健保署今年3月與廣達電腦董事長林百里共同簽署合作備忘錄，由廣達捐贈健保署「人工智慧醫療雲運算整合平台（QOCA AIM）」系統，並客製化打造有利健保資料加值的環境。
林百里表示，全民健保開辦至今25年來，所累積的就醫紀錄結構化資料高達649億筆、醫療影像自107年收

▲10月消費者信心指數調查中，投資股市時機指標下降最多。（圖／記者屠惠剛攝）
記者余弦妙／台北報導
中央大學台灣經濟發展研究中心今（27）日發布10月消費者信心指數(CCI)調查的總數為71.1點，與上月相較下降0.5點。本次的六項分項指標中，三項指標上升 (物價水準、國內經濟景氣、購買耐久財) ，三項指標下降(家庭經濟狀況、就業機會、投資股市時機) ；上升幅度最多的指標是「未來半年國內物價水準」 。
根據調結果顯示，上升幅度最多的指標是「未來半年國內物價水準」，本次調查結果為42.05點，較上個月調查結果上升3.05點，其次為「未來半年購買耐久性財貨時機」，本月調查結果為122.3點，較上個月上升2.45點，再來則是，「未來半年國內經濟景氣」，本月調查結果為83.8點，與上個月調查結果相較上升0.8點。
而在下降指標中，下降最多的則是「未來半年投資股票時機」，本月調查為30點，較上個月調查結果下降5.3點，接下來則是「未來半年家庭經濟狀況」，本次調查結果為81.65點，與上月的調查結果相較下降2.15點，最後為「未來半年國內就業機會」，本月調查結果為66.8點，與上個月調查結果相較下降1.85點。
----------------------------------------------------------------------

[45] 約妹子逛夜市花600元！　他怒「沒有AA制」遭網嗆爆：憑實力單身

▲原PO和妹子一起去逛夜市。（示意圖／Pixabay）
記者施怡妏／綜合報導
一名男網友分享，與聊了一天的妹子約見面，兩人一起去逛夜市，沒想到妹子食量驚人，夜市從頭吃到尾總共吃了600多元，還全都由男方買單，吃完後便直接搭捷運結束這一趟約會，他傻眼直呼「人生好難」。對此，許多網友留言嗆，「600元也敢拿出來說嘴？」
▲原PO傻眼妹子竟然吃飽飯就閃人。（示意圖／取自免費圖庫Pexels）
貼文發出後，遭到許多網友嗆爆「憑實力單身」，「600多元很便宜阿，男人大氣一點，這種小錢就別計較了」、「你知道嗎，我光帶小孩去夜市，走一圈都不只600元了」、「才第一次見面，你想怎樣」、「有人陪逛街便宜了」、「600元還不AA這種話還是放在心裡吧」、「終於明白你為什麼沒有女朋友了」、「傳播不止600元」。
--------------------------------

▲黃金價格在每盎司1900美元附近盤整。（圖／路透）
記者林妤柔／綜合報導
黃金價格在每盎司1900美元附近盤整，隨著美國總統大選到來，投資者希望財政刺激方案能促使黃金反彈。截至稍早，12月黃金期貨價格上漲0.02%，至每盎司1906美元。
IG Group高級外匯分析師Christopher Vecchio認為，如果拜登獲勝，但參議院席位仍以共和黨佔多數，將很難通過新的刺激措施，若民主黨佔多數，新紓困方案就能推動，使美元下跌，帶動股票、黃金上漲。
Cooper認為，在美國總統大選前，現貨市場一直疲軟，第三季可能成為央行10年來最弱的黃金購買季。
其他貴金屬部分，截至稍早，12月白銀價格上漲0.02%至每盎司24.4美元；1月鉑金價格上漲0.08%至每盎司878.1美元；12月鈀金上漲0.23%至每盎司2374.1美元。
----------------------------------------------------------------------

[53] 股匯不同調！新台幣大升近3角　來到28.61元

▲新台幣開盤貶值1.3分。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（27）日開盤貶值1.3分，以28.915元兌1美元開出，隨著熱錢湧入，新台幣大升2.92角，來到28.61元。
新台幣匯率昨（26）日收在28.902元兌1美元，今天以28.915元兌1美元開出。
----------------------------------------------------------------------

[54] 台股下跌逾80點！12900關卡失守　台積電跌0.67%

▲台股今開盤下跌。（圖／記者屠惠剛攝）
記者林妤柔／綜合報導
美國確診數創新高，使疫情再次急墜之下，加上新一輪紓困方案卡關，投資前景受挫，道瓊一度重挫逾600點。台股今（27）日開在12867.77點，跌84.7點，指數暫收12829.78點。
蘋概三王中，台積電（2330）今日以447.5元開出、跌0.67%；大立光（3008）以3160元開出、跌1.1%；鴻海（2317）以79.9元開出、跌0.5%。
聯發科（2454）以674元開出，原本下跌0.3%，目前翻紅上漲0.45%。
記憶體廠旺宏（2337）昨（26）日舉行法說會，受惠於中芯轉單

▲夏普旗下可購樂平台率先販售南六雙鋼印口罩。（圖／夏普提供）
記者姚惠茹／台北報導
「可購樂CocoroLife」為鴻海旗下夏普主導的電商平台，可購樂平台提供橫跨食、衣、住、行、娛樂及健康需求商品，超過萬件商品，只要民眾在2016年至2020年共5年間曾為鴻海/鴻準/群創的股東，即是可購樂尊榮的VIP股東身份，立即登入可購樂平台享有上萬商品優惠。
----------------------------------------------------------------------

[67] 亞泰金屬年底前掛牌上櫃　董座鄒貴銓：明年接單已滿

▲PCB設備廠亞泰金屬董事長鄒貴銓。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
PCB設備廠亞泰金屬（6727）即將在明（27）日舉辦上櫃前業績發表會，董事長鄒貴銓表示，亞泰金屬受惠5G與電動車產業發展，目前產能已滿，因此明年初將在楊梅擴建新廠，預計後年完工投產，而且明年接單已滿，法人看好，今明兩年的營收可望逐年成長10%，預計年底前掛牌上櫃。
鄒貴銓表示，亞泰金屬創立於1973年，塗佈設備最早應用於膠帶生產，1986年擴展新事業，研發PCB上游材料CCL含浸設備，主力產品為印刷電路板基材製程設備(含浸機、貼合機)、薄膜製程設備(塗佈機、能源基材製程設備)，也應用於軟板(FCCL)、被動元件、奈米銀絲觸控、碳纖維及光學領域。
鄒貴銓指出，相較傳統設備業，可廣泛應用於捲對捲(R2R)精密塗佈跨領域的產業，產業終端應用已遍及電子零組件產業、塑膠產業、能源產業、醫療產業、光電產業、交通產業、建築產業至航太產業，明年訂單有3成在台灣，主要受到中美貿易戰影響，台商回流趨勢明顯。
亞泰金屬總經理黃源財表示，隨著5G與電動車產業高速發展，帶動PCB上游材料廠的擴廠需求，面對日系與歐系大廠的競爭，在雙方設備技術能力伯仲之間情況下，亞泰金屬憑藉在價格及售後服務上的優勢，使得競爭力優於同業廠商，也逐步搶佔市場佔有率。
黃源財指出，全球5G高頻高速CCL供應商有75%是亞泰金屬的客戶，並因應客戶新製程需求，推出高溫、無塵等新型設備，所生產的含浸設備可滿足生產超薄布種的需求，訂單能見度已排到明年，而且接單呈現全滿的狀態。
黃源財分享，由於產能受限，現有產線主要用以生產直立式含浸設備的產能已滿，為更完善提供客戶良好的服務及生產水平設備訂單，亞泰金

▲美股道瓊期出現重跌。（圖／路透）
記者余弦妙／台北報導
美國三大期貨於台灣時間26日15時26分左右出現下跌，道瓊期暴跌超過300點，那斯達克100期則是跌了將近100點，而標普500期小跌34點。
道瓊期指截至台灣時間15：26，為27881.0點，下跌308點或1.09%。
另外，那斯達克期100指下跌點94.38點或0.81%，為11569.12點。
標普500期指則小跌34.87或1.01%，來到3416.88點。
----------------------------------------------------------------------

[79] 帶手機就好！好市多「電子會員卡、Costco Pay」上線：9步驟開通

▲好市多新增「電子會員卡」功能。（圖／翻攝自「COSTCO好市多」APP）
記者曾筠淇／綜合報導
美式賣場好市多（Costco）深受台灣消費者喜愛，為了維持品質，如果想要入內購物，店家都會要求顧客出示會員卡。然而，有些民眾卻會在到了現場後，才忽然發現會員卡沒帶出門；對此，好市多目前已經更新APP，推出「電子會員卡」的功能，未來就算忘記攜帶實體卡也無妨。
「COSTCO好市多」APP全新改版，新增了「電子會員卡」、「Costco Pay」、「好事在這編」以及「黑鑽卡2%回饋金及升等費用查詢」等重要功能。根據官方的更新說明指出，「Costco電子會員卡」只要透過手機認證並註冊，設定完成後攜帶身分證明文件至會員部開通，日後進入賣場前，出示APP的電子會員卡即等同實體卡片。
▲下載APP並點選「會員登入」。（圖／翻攝自「COSTCO好市多」APP）
▲▼好市多電子會員卡申辦步驟。（圖／翻攝自「COSTCO好市多」APP）
▲使用前必須先至會員部開通。（圖／翻攝自「COSTCO好市多」APP）
官網更新說明也提到，Costco Pay可綁定Costco聯名信用卡或國泰世華銀行所發行的Visa簽帳金融卡，除了能夠將發票存入載具外，未來還可以直接使用手機結帳，方便度再提升。
▼好市多還推出「Costco Pay」。（圖／翻攝自「COSTCO好市多」APP）
----------------------------------------------------------------------

[80] MagSafe、Light

● 吳嘉隆／總體經濟學家
美國大選實在太扣人心弦，所以有些人可能想從股票市來找線索，做一下推測￼，因為我們總是相信，金融市場對政治變動是高度敏感的，覺得市場肯定知道的比我多。
拜登在兩次總統大選辯論中，￼有提到一些與經濟相關的議題，￼例如：
（1）為了環保而把頁岩油產業拿掉，改推太陽能與風能￼。
（2）取消川普的減稅，改成加稅。
（可是，這對美國企業界與金融界是不友善的。）￼
（3）提高每小時最低工資到15美元。
（4）把現有的1100萬非法移民合法化。
（5）取消川普對中國出口產品加的關稅￼
（6）重新加入世界衛生組織。
（可是，以上兩條等於是否定川普對中國的強硬態度，這樣一來，其他國家怎麼敢站在美國這邊來對抗中共？這等於是美國開始向中共投降，讓出世界領導地位，讓盟友無所適從，￼拜登將沒有辦法治理這樣的美國。）
▲ 學者指出，拜登的經濟政策會使金融市場大跌。（圖／路透）
民主黨不懂經濟￼￼
經過這樣的解說，我發現民主黨基本上不懂經濟，會把事情搞砸，然後說「在這樣的情況下，我沒辦法，只好這樣了」。
柯林頓時代有把美國經濟搞好，但是他的路線其實是走共和黨的經濟思路，他用的聯準會主席也是共和黨人格林斯潘￼。
講回來，那麼現在美國的股市怎麼樣看美國大選呢？
這兩個指數都是在相對高檔盤旋，並沒有任何恐慌情緒出來。
這個行情走高代表市場願意承擔風險，投機性比較高，這個行情走低代表市場有避險情緒，比較保守。
從這裡我做出一個合理的推測，就是美國金融市場基本上是認為川普會連任的。
► Her和她 女孩想要的都在這
熱門點閱》
►  王高成／川普未透過辯論扳回劣勢　拜登較受中間選民青睞
►  吳嘉隆／習近平會在美大選後　對台灣發動奇襲嗎？
►  陳方隅／台裔打入拜登決策圈　實質影響外交政策走向？
►  美選辯論速記／拜登打破失言刻板印象　川普修正搶話形象有利好感度
----------------------------------------------------------------------

[91] 15年前「300萬存3支股」進帳千萬　結局揭曉超意外：台積電賺最少

▲示意圖，與本文無關。（圖／記者屠惠剛攝）
記者周亭瑋／綜合報導
華倫老師日前指出，假設你在2006年1月1日拿出300萬元，分別買進大統益、中華食及台積電的股票，接著封鎖營業員、不看第四台等，杜絕所